## This is notebook is mainly used for Coursera Capstone Project
The notebook is also available on github https://github.com/europateam/Coursera_Capstone.git

In [3]:
import pandas as pd
import numpy as np
import babel.numbers
import decimal

# library to handle requests
import requests 

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 
import json

!pip install geopy
from geopy.geocoders import Nominatim

# Matplotlib and associated plotting modules
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
mpl.style.use(['ggplot']) # optional: for ggplot-like style

!pip install folium
import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#### Download LA Census Tract Locations

In [ ]:
# GEOID  to connect neighbor to CENSUS DATA 
#https://usc.data.socrata.com/Los-Angeles/Census-Tract-Locations-LA-/atat-mmad
#Census Tract Locations (LA)
    
df_geo = pd.read_csv('https://usc.data.socrata.com/resource/atat-mmad.csv')

# clean-up dataframe
df_geo = df_geo.drop(['tract','location'],axis=1)
df_geo.columns = ['geoid','tract','neighborhood','latitude','longitude']
df_geo['geoid'] = df_geo.geoid.str.replace('1400000US','')
df_geo.geoid = df_geo.geoid.astype('int64')



CENTRAL_LA = ['Arlington Heights','Beverly Grove','Carthay','Chinatown','Downtown','East Hollywood','Echo Park','Elysian Park','Elysian Valley'
,'Faircrest Heights', 'Griffith Park', 'Hancock Park','Harvard Heights','Hollywood','Hollywood Hills','Hollywood Hills West'
,'Koreatown','Larchmont','Los Feliz','Mid-City','Mid-Wilshire','Pico-Union','Silver Lake','West Hollywood'
,'Westlake','Windsor Square']

# filtered by CENTRAL LA neighborhood
df_geo = df_geo[df_geo['neighborhood'].isin (CENTRAL_LA)]

df_geo = df_geo.reset_index()
df_geo.drop(['index','latitude','longitude'],axis=1,inplace=True)
#df_geo.head() 

#### Create Neighborhood Median Income Dataframe

In [ ]:
# Living Wage (2017)
# https://data.lacounty.gov/Sustainability/Living-Wage-2017-/f5es-twcr
df_wage = pd.read_csv('https://data.lacounty.gov/resource/f5es-twcr.csv')

#clean-up dataframe
df_wage = df_wage[['geoid','tractce','medincome']]
df_wage.rename(columns = {'tractce':'tract','medincome': 'income'},inplace=True)

#df_wage.shape
#df_wage.head()

#### Create Neighborhood population Dataframe

In [ ]:
# LA County Estimated Population (2016)
# https://dev.socrata.com/foundry/data.lacounty.gov/ai64-dnh8
# County of Los Angeles Estimated Population by Census Tract, City Name, Ethnicity, Gender & Age Group: CY 2016

url = 'https://data.lacounty.gov/resource/ai64-dnh8.csv'

df_pop = pd.read_csv(url)

#clean-up dataframe
df_pop.drop(['fips','cityname','service_area'],axis=1,inplace=True)
df_pop.rename(columns = {'census_tract':'tract'},inplace=True)
df_pop['total_pop'] = df_pop[['male','female']].sum(axis=1)

#df_pop.shape
#df_pop.head()

#### Create a combined Neighborhood Median Income & Population Dataframe 

In [ ]:
# add geoid to income and popuplation dataframe. 
# It will be used to draw heat map by income / population
df_la_income = pd.merge(df_geo[['geoid','neighborhood']], df_wage, how='left', left_on='geoid', right_on='geoid')

df_la_income_pop = pd.merge(df_la_income, df_pop [['tract','white','asian','latino','total_pop']], how='left', left_on='tract', right_on='tract')

#df_la_income_pop.head()

In [ ]:
# create a rolledup income and population by neighborhood. 
# It will be used to plot income / population of neighborhood

df_neighborhood_income_pop = df_la_income_pop.groupby('neighborhood').agg(
                                {
                                     'total_pop':sum,   
                                     'white': sum,
                                     'asian': sum,
                                     'latino': sum,
                                     'income': "mean"
                                }
                            ).reset_index()

df_neighborhood_income_pop.rename(columns = {'white':'white_pop','asian':'asian_pop','latino':'latino_pop'},inplace=True)

#df_neighborhood_income_pop.head()

In [ ]:
# add latitude and longitute to each neighborhood
lat =[]
long = []
for neighbor in df_neighborhood_income_pop.neighborhood:
    address = '{}, Los Angeles CA'.format(neighbor)

    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    lat.append(location.latitude)
    long.append(location.longitude)

    
df_neighborhood_income_pop['latitude'] = lat
df_neighborhood_income_pop['longitude'] = long

#df_neighborhood_income_pop.head()

#### Visualize neighborhood median income / population of neighborhoods

In [ ]:
def plotBarGraph(df_data,title,y_value,y_label,bar_color, legend_on):
    
    df_data[y_value].plot(kind='bar', color= bar_color, stacked=False, figsize=(16, 6))
    plt.xlabel('Neighborhood',fontsize=16) # add to x-label to the plot
    plt.ylabel(y_label,fontsize=16) # add y-label to the plot
    plt.title(title, fontsize=24) # add title to the plot

    if (legend_on):
        colors = {'population':'blue', 'median income':'red'}         
        labels = list(colors.keys())
        handles = [plt.Rectangle((0,0),1,1, color=colors[label]) for label in labels]
        plt.legend(handles, labels)
    else:
        plt.legend().remove()
        
    plt.show()

In [ ]:
df_data = df_neighborhood_income_pop
df_data= df_data.set_index('neighborhood')
plotBarGraph(df_data,'Central Los Angeles Neighborhood Population - Median Income Ratio',['total_pop','income'],'Population / Median Income',['blue', 'red'], 1 )

In [ ]:

df_data[['white_pop','asian_pop','latino_pop']].plot(kind='bar', stacked=True, figsize=(14, 6)).legend(['White populate','Asian population','Latino population'],loc='upper center', ncol=3, title="")

#### Create visualization of neighborhood by household and income
#label_names = ['White populate','Asian population','Latino population']
#plt.legend(label_names)
plt.xlabel('Neighborhood') # add to x-label to the plot
plt.ylabel('Population') # add y-label to the plot
plt.title('Central Los Angeles Neighborhood Population by Race') # add title to the plot

plt.show()


In [ ]:
plotBarGraph(df_data,'Central Los Angeles Neighborhood Population',['total_pop'],'Population',['blue'], 0)

In [ ]:
plotBarGraph(df_data,'Central Los Angeles Neighborhood Median Income',['income'],'Median Income',['red'],0 )

In [ ]:
# choropleth map
def create_HeatMap(title,legendname, df_data, map_attribute, df_markers):
    la_geo = r'https://opendata.arcgis.com/datasets/d6c55385a0e749519f238b77135eafac_0.geojson'
    # create a plain LA map
    central_la = [34.079870, -118.311338]

    la_map = folium.Map(location=central_la, zoom_start=12,tiles='cartodbpositron')

    # generate choropleth map
    folium.Choropleth(
        geo_data=la_geo,
        data=df_data,
        columns=['neighborhood', map_attribute],
        key_on='feature.properties.name',
        fill_color='YlOrRd', 
        fill_opacity=0.7, 
        line_opacity=0.2,
        highlight=True,
        line_color='black',
        nan_fill_color="white",
        legend_name= legendname
    ).add_to(la_map)

    title_html = '''<h3 align="center" style="font-size:16px"><b>{}</b></h3>'''.format(title)   
    la_map.get_root().html.add_child(folium.Element(title_html))

   
    if not df_markers.empty:
        
        # set color scheme for the clusters
        x = np.arange(kclusters)
        ys = [i + x + (i*x)**2 for i in range(kclusters)]
        colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
        rainbow = [colors.rgb2hex(i) for i in colors_array]


        # add markers to the map
        for lat, lon, poi, cluster, cluster_label, income, total_pop, white_pop, asian_pop, latino_pop in zip(df_markers['latitude'],\
                                                             df_markers['longitude'], df_markers['neighborhood'],\
                                                             df_markers['cluster'], df_markers['cluster_label'],\
                                                             df_markers['income'], df_markers['total_pop'],\
                                                             df_markers['white_pop'], df_markers['asian_pop'],\
                                                             df_markers['latino_pop']):

            # build label info
            
            label_text = '<h4 align="center">' + str(poi) + '</h4><b>Cluster:</b> ' + cluster_label + \
                            '<br><b>Median income:</b> $' + str(income) + \
                            '<br><b>Population:</b> ' + str(total_pop) + \
                            '<br><b>White:</b> ' + str(white_pop) + ' - <b>Asian:</b> ' + \
                            str(asian_pop) + ' - <b>Latino:</b> ' + str(latino_pop)   
            
            label_info = folium.Html(label_text,script=True)
            label = folium.Popup(label_info, max_width=400)

            folium.CircleMarker(
                [lat, lon],
                radius=6,
                popup=label,
                color=rainbow[cluster-1],
                fill=True,
                fill_color=rainbow[cluster-1],
                fill_opacity=0.7).add_to(la_map)

    # display map
    return la_map

In [ ]:
# remove missing data
df_neighborhood_income_pop = df_neighborhood_income_pop.loc[(df_neighborhood_income_pop.total_pop != 0)]

create_HeatMap("Central LA Neighborhood Population","Population", df_neighborhood_income_pop, 'total_pop',pd.DataFrame())

#### Foursquare - explore restaurant around LA CENTRAL neighborhoods

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, query, radius=750):
    
    venues_list=[]
    nearby_venues = None
    for name, lat, lng in zip(names, latitudes, longitudes):

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&limit={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            query,
            50,
            radius, 
            LIMIT)

        # make the GET request
        
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']

            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])

            nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
            nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']

        except KeyError as ke:
            pass
        
    return(nearby_venues)


In [ ]:
# explore food venues 
la_venues = getNearbyVenues(df_neighborhood_income_pop.neighborhood, df_neighborhood_income_pop.latitude,df_neighborhood_income_pop.longitude,'food')

In [ ]:
# classify venue category into 3 food cateogries: asian, latin, amerian/european
la_cuisine = la_venues
la_cuisine['cuisine_type'] = 'Un-Classified'

# Mexican food 
searchfor = ['Mexican','Argentinian','Brazilian','Latin','Peruvian','Spanish','Cuban'
             ,'South American']
la_cuisine.loc[(la_cuisine['Venue Category'].str.contains('|'.join(searchfor))),'cuisine_type'] = 'Latin American Food'


#la_asian
searchfor = ['Korean','Chinese','Sushi','Asian','Thai','Vietnamese','Hotpot','Dim Sum'
             ,'Indian','Noodle','Ramen','Dumpling','Filipino','Japanese','Satay','Taiwanese'
             ,'Poke','Middle Eastern','Donburi Restaurant','Cantonese','Shabu-Shabu']
la_cuisine.loc[(la_cuisine['Venue Category'].str.contains('|'.join(searchfor))),'cuisine_type'] = 'Asian Food'


#la_american
searchfor = ['Creperie','New American','French','Italian','Steakhouse','American Restaurant','Cajun'
             ,'Mediterranean','Seafood','BBQ Joint','German','Gastropub','Irish','Greek']
la_cuisine.loc[(la_cuisine['Venue Category'].str.contains('|'.join(searchfor))),'cuisine_type'] = 'American/Eurpoean Food'


# merge 3 food categories

# clean up la_venue_region
la_cuisine = la_cuisine[la_cuisine.cuisine_type!='Un-Classified']
la_neighborhood_cuisine = la_cuisine[['Neighborhood','cuisine_type']]

#la_neighborhood_cuisine

#### Analyze venues for each Neighborhood

In [ ]:
# one hot encoding
la_cuisine_onehot = pd.get_dummies(la_neighborhood_cuisine[['cuisine_type']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
la_cuisine_onehot['Neighborhood'] = la_neighborhood_cuisine['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [la_cuisine_onehot.columns[-1]] + list(la_cuisine_onehot.columns[:-1])
la_cuisine_onehot = la_cuisine_onehot[fixed_columns]

#la_cuisine_onehot.tail(30)

la_neighborhood_cuisine_grouped = la_cuisine_onehot.groupby('Neighborhood').mean().reset_index()
#la_neighborhood_cuisine_grouped

In [ ]:
# function returns most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_cuisine_sorted = pd.DataFrame(columns=columns)
neighborhoods_cuisine_sorted['Neighborhood'] = la_neighborhood_cuisine_grouped['Neighborhood']

for ind in np.arange(la_neighborhood_cuisine_grouped.shape[0]):
    neighborhoods_cuisine_sorted.iloc[ind, 1:] = return_most_common_venues(la_neighborhood_cuisine_grouped.iloc[ind, :], num_top_venues)

#neighborhoods_cuisine_sorted

#### Using Elbow method to determine the optimal K-Mean

In [ ]:
la_cuisine_grouped_clustering = la_neighborhood_cuisine_grouped.drop('Neighborhood', 1)
 

distortions = [] 
inertias = [] 
mapping1 = {} 
mapping2 = {} 
K = range(1,10) 
  
for k in K: 
    #Building and fitting the model 
    kmeanModel = KMeans(n_clusters=k).fit(la_cuisine_grouped_clustering) 
    kmeanModel.fit(la_cuisine_grouped_clustering)     
     
    distortions.append(sum(np.min(cdist(la_cuisine_grouped_clustering, kmeanModel.cluster_centers_, 
                      'euclidean'),axis=1)) / la_cuisine_grouped_clustering.shape[0]) 
    inertias.append(kmeanModel.inertia_) 
  
    mapping1[k] = sum(np.min(cdist(la_cuisine_grouped_clustering, kmeanModel.cluster_centers_, 
                 'euclidean'),axis=1)) / la_cuisine_grouped_clustering.shape[0] 
    mapping2[k] = kmeanModel.inertia_ 
    

#for key,val in mapping1.items(): 
#    print(str(key)+' : '+str(val)) 
    
plt.plot(K, distortions, 'bx-') 
plt.xlabel('Values of K') 
plt.ylabel('Distortion') 
plt.title('The Elbow Method using Distortion') 
plt.show()

In [ ]:
#for key,val in mapping2.items(): 
#    print(str(key)+' : '+str(val)) 
    
plt.plot(K, inertias, 'bx-') 
plt.xlabel('Values of K') 
plt.ylabel('Inertia') 
plt.title('The Elbow Method using Inertia') 
plt.show() 

#### Cluster Neighborhoods

In [ ]:
# based on the Elbow method to determine the optimal K-Mean
# set Kmean = 4
kclusters = 4

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(la_cuisine_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 


In [ ]:
la_neighborhoods_cuisine_merged = pd.merge(df_neighborhood_income_pop, neighborhoods_cuisine_sorted, 
                  how='inner', left_on='neighborhood', right_on='Neighborhood')

la_neighborhoods_cuisine_merged = la_neighborhoods_cuisine_merged.drop(['Neighborhood'],axis=1)

# channge datatype
la_neighborhoods_cuisine_merged.income = round(la_neighborhoods_cuisine_merged.income,2)
la_neighborhoods_cuisine_merged.total_pop = la_neighborhoods_cuisine_merged.total_pop.astype(int)
la_neighborhoods_cuisine_merged.white_pop = la_neighborhoods_cuisine_merged.white_pop.astype(int)
la_neighborhoods_cuisine_merged.asian_pop = la_neighborhoods_cuisine_merged.asian_pop.astype(int)
la_neighborhoods_cuisine_merged.latino_pop = la_neighborhoods_cuisine_merged.latino_pop.astype(int)

# add clustering labels
la_neighborhoods_cuisine_merged.insert(0,'cluster',0)
la_neighborhoods_cuisine_merged.insert(1,'cluster_label','')
la_neighborhoods_cuisine_merged['cluster'] =  kmeans.labels_

#la_neighborhoods_cuisine_merged.head() # check the last columns!



#### Examine Cluster

In [ ]:
la_neighborhoods_cuisine_merged[['cluster','neighborhood','1st Most Common Venue','2nd Most Common Venue','3rd Most Common Venue']].head()

In [ ]:
# visualize cuisine type distribution in each cluster
df_data = la_neighborhoods_cuisine_merged[['cluster','1st Most Common Venue']]
df_data.insert(2, 'count',1)
df_data.columns =['cluster','cuisine','count']

df_data

df_pivot = pd.pivot_table(
    df_data, 
    values="count",
    index="cluster",
    columns="cuisine", 
    aggfunc=np.sum
)

ax = df_pivot.plot(kind='bar',title='Number of Cuisine Type in Each Cluster')
fig = ax.get_figure()
fig.set_size_inches(10,4)
ax.set_xlabel("Cluster")
ax.set_ylabel("Count")
ax.legend()
plt.show()


##### Create label for cluster after examine the clusters

In [ ]:
# assign cluster label
la_neighborhoods_cuisine_merged.loc[la_neighborhoods_cuisine_merged.cluster == 0,['cluster_label']] = 'Asian Cuisine Dominant'
la_neighborhoods_cuisine_merged.loc[la_neighborhoods_cuisine_merged.cluster == 1,['cluster_label']] = 'American/European Cuisine Dominant' 
la_neighborhoods_cuisine_merged.loc[la_neighborhoods_cuisine_merged.cluster == 2,['cluster_label']] = 'Mixture of Asian - American Cuisine' 
la_neighborhoods_cuisine_merged.loc[la_neighborhoods_cuisine_merged.cluster == 3,['cluster_label']] = 'Latin American Cuisine Dominant'
#la_neighborhoods_cuisine_merged.head()

In [ ]:
# cleanup unused fields
la_neighborhoods_cuisine_merged.drop(['1st Most Common Venue','2nd Most Common Venue','3rd Most Common Venue'],axis=1,inplace=True)

In [ ]:
la_neighborhoods_cuisine_merged.head()

#### Create cluster visualization

In [ ]:
#create_map(title,legendname, df_data, map_attribute):
create_HeatMap ('Cuisine Clusters by Median Income', 'Median Income', df_neighborhood_income_pop, 'income', la_neighborhoods_cuisine_merged )

In [ ]:
create_HeatMap ('Cuisine Clusters by Population', 'Population', df_neighborhood_income_pop, 'total_pop', la_neighborhoods_cuisine_merged )

In [ ]:
create_HeatMap ('Cuisine Clusters by Latino Population', 'Latino Population', df_neighborhood_income_pop, 'latino_pop', la_neighborhoods_cuisine_merged )

In [ ]:
create_HeatMap ('Cuisine Clusters by Asian Population', 'Asian Population', df_neighborhood_income_pop, 'asian_pop', la_neighborhoods_cuisine_merged )

In [ ]:
create_HeatMap ('Cuisine Clusters by White Population', 'White Population', df_neighborhood_income_pop, 'white_pop', la_neighborhoods_cuisine_merged )